In [1]:
import pandas as pd

df1 = pd.read_csv("Batch_2_train_labels_clean.csv")
df2 = pd.read_csv("Batch_1_train_labels_clean.csv")

df3 = pd.read_excel("image_labels_batch_3.xlsx")
df4 = pd.read_excel("image_labels_batch_4.xlsx")




In [2]:


df = pd.concat([df1, df2, df3], ignore_index=True)



In [3]:
print(df)

                                  image_name           category  \
0       image_874729307_product_74244625.jpg          PC Gaming   
1       image_874730792_product_49132716.jpg           Nintendo   
2       image_874731199_product_74093483.jpg        PlayStation   
3       image_874731370_product_49948298.jpg           Nintendo   
4      image_881487935_product_126111824.jpg           Nintendo   
..                                       ...                ...   
346  image_1100063096_product_1711731907.jpg          PC Gaming   
347  image_1100063685_product_1711736000.jpg          PC Gaming   
348  image_1100064358_product_1711734478.jpg          PC Gaming   
349  image_1119523636_product_2000192031.jpg  Réalité Virtuelle   
350  image_1139162966_product_2379124886.jpg           Nintendo   

               subcategory  
0                  Jeux PC  
1    Jeux Game Boy Advance  
2                 Jeux PS3  
3    Jeux Game Boy Advance  
4         Jeux Nintendo DS  
..                   

In [5]:
image_dir = "batchs_équilibrés\\batch_entrainement - Copie\\"


In [6]:
import os

# Liste de toutes les images du dossier
all_images = set(os.listdir(image_dir))  # Utilisation d'un set pour une recherche rapide


In [7]:
# Liste des images attendues selon le DataFrame
expected_images = set(df['image_name'].astype(str))  # Convertir en set pour optimisation

# Filtrer les images existantes
valid_images = expected_images.intersection(all_images)  # Conserver uniquement celles qui existent

print(f"📸 {len(valid_images)} images valides trouvées sur {len(expected_images)} attendues.")


📸 266 images valides trouvées sur 351 attendues.


In [8]:
for img in all_images - valid_images:  # Différence des ensembles
    img_path = os.path.join(image_dir, img)
    os.remove(img_path)
    print(f"🗑️ Image supprimée : {img}")


🗑️ Image supprimée : image_1160912325_product_2726077368.jpg
🗑️ Image supprimée : image_1236934329_product_3716121060.jpg
🗑️ Image supprimée : image_1199617908_product_3233258914.jpg
🗑️ Image supprimée : image_1100081873_product_1711736649.jpg
🗑️ Image supprimée : image_1168313731_product_2856775404.jpg
🗑️ Image supprimée : image_1100066257_product_1711735869.jpg
🗑️ Image supprimée : image_966604797_product_252504330.jpg
🗑️ Image supprimée : image_1108737984_product_1878218298.jpg
🗑️ Image supprimée : image_1100076852_product_1711728854.jpg
🗑️ Image supprimée : image_1100065974_product_1711736557.jpg
🗑️ Image supprimée : image_1236934931_product_3716121410.jpg
🗑️ Image supprimée : image_1100062722_product_1711729209.jpg
🗑️ Image supprimée : image_1176954743_product_2965041516.jpg
🗑️ Image supprimée : image_1240554261_product_3748316863.jpg
🗑️ Image supprimée : image_1100066292_product_1711728922.jpg
🗑️ Image supprimée : image_927527365_product_180498882.jpg
🗑️ Image supprimée : image_1

In [10]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

def load_image(img_name, label):
    try:
        # Utiliser tf.py_function pour s'assurer que l'on travaille avec une chaîne et non un tenseur
        img_name_str = tf.py_function(lambda x: x.numpy().decode('utf-8'), [img_name], Tout=tf.string)

        img_path = tf.strings.join([image_dir, img_name_str])  # Construire le chemin complet
        img = tf.io.read_file(img_path)  # Lire l'image
        img = tf.image.decode_jpeg(img, channels=3)  # Décoder l'image
        img = tf.image.resize(img, [224, 224])  # Redimensionner l'image
        img = img / 255.0  # Normaliser l'image
        return img, label

    except Exception as e:
        print(f"Erreur lors du chargement de l'image {img_name}: {e}")
        return tf.zeros([224, 224, 3]), label  # Retourner une image vide au lieu de None

In [11]:
# Ne garder que les noms d'images qui existent physiquement
df_filtered = df[df['image_name'].isin(valid_images)]

# Convertir en TensorFlow Dataset
image_names = df_filtered["image_name"].values
labels = df_filtered["category_encoded"].values

dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))
dataset = dataset.map(lambda x, y: load_image(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [13]:
import numpy as np

# Convertir les noms d'images et labels en TensorFlow Dataset
image_names = df["image_name"].values
labels = df["category_encoded"].values

# Créer un dataset TensorFlow à partir des chemins d'images et des labels
dataset = tf.data.Dataset.from_tensor_slices((image_names, labels))

# Appliquer la fonction de chargement d'image et label avec map
dataset = dataset.map(lambda x, y: load_image(x, y))

# Diviser en batchs, mélanger et pré-charger pour améliorer les performances
dataset = dataset.batch(32).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

# Modèle simplifié pour tester
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(np.unique(labels)), activation='softmax')
])

# Compiler et tester l'entraînement avec un modèle plus simple
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(dataset, epochs=1)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 192: invalid continuation byte

In [14]:
# Afficher les premières lignes pour vérifier les noms d'images
print(df['image_name'].head())


# Vérifier les noms d'images pour des caractères spéciaux ou invisibles
for name in df['image_name']:
    try:
        name = str(name).encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    except UnicodeDecodeError:
        print(f"Erreur avec le nom de fichier : {name}")


0     image_874729307_product_74244625.jpg
1     image_874730792_product_49132716.jpg
2     image_874731199_product_74093483.jpg
3     image_874731370_product_49948298.jpg
4    image_881487935_product_126111824.jpg
Name: image_name, dtype: object
